In [1]:
# Load LSTM network and generate text
import sys
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

/usr/lib/python3/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# load ascii text and covert to lowercase
filename = "alice.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [3]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [25]:
# summarize the loaded data
n_chars = int(len(raw_text)/50)
n_vocab = len(chars)
print( "Total Characters: ", n_chars)
print( "Total Vocab: ", n_vocab)

Total Characters:  2893
Total Vocab:  47


In [26]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print( "Total Patterns: ", n_patterns)

Total Patterns:  2793


In [27]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [28]:
# X.SHAPE IS THE LENGTH OF THE DESIRE RETURN IF I WANT TO RETURN 
# define the LSTM model
model = Sequential()
# Adding a return_sequence allow the next layer to have a 3D sequence input
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [102]:
X.shape[2]

1

In [29]:
# load the network weights
# filename = "weights-improvement-19-1.9435.hdf5"
# model.load_weights(filename)

filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger2.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [30]:
# fit the model
model.fit(X, y, epochs=70, batch_size=64, callbacks=callbacks_list)

Epoch 1/70
2793/2793 [==============================] - 31s 11ms/step - loss: 3.1679

Epoch 00001: loss improved from inf to 3.16793, saving model to weights-improvement-01-3.1679-bigger2.hdf5
Epoch 2/70
2793/2793 [==============================] - 29s 10ms/step - loss: 3.0378

Epoch 00002: loss improved from 3.16793 to 3.03778, saving model to weights-improvement-02-3.0378-bigger2.hdf5
Epoch 3/70
2793/2793 [==============================] - 29s 10ms/step - loss: 3.0348

Epoch 00003: loss improved from 3.03778 to 3.03484, saving model to weights-improvement-03-3.0348-bigger2.hdf5
Epoch 4/70
2793/2793 [==============================] - 29s 10ms/step - loss: 3.0268

Epoch 00004: loss improved from 3.03484 to 3.02677, saving model to weights-improvement-04-3.0268-bigger2.hdf5
Epoch 5/70
2793/2793 [==============================] - 29s 10ms/step - loss: 3.0189

Epoch 00005: loss improved from 3.02677 to 3.01893, saving model to weights-improvement-05-3.0189-bigger2.hdf5
Epoch 6/70
2793/279

2793/2793 [==============================] - 29s 10ms/step - loss: 1.2825

Epoch 00043: loss improved from 1.38601 to 1.28254, saving model to weights-improvement-43-1.2825-bigger2.hdf5
Epoch 44/70
2793/2793 [==============================] - 29s 10ms/step - loss: 1.2013

Epoch 00044: loss improved from 1.28254 to 1.20132, saving model to weights-improvement-44-1.2013-bigger2.hdf5
Epoch 45/70
2793/2793 [==============================] - 29s 10ms/step - loss: 1.1047

Epoch 00045: loss improved from 1.20132 to 1.10473, saving model to weights-improvement-45-1.1047-bigger2.hdf5
Epoch 46/70
2793/2793 [==============================] - 29s 10ms/step - loss: 1.0183

Epoch 00046: loss improved from 1.10473 to 1.01828, saving model to weights-improvement-46-1.0183-bigger2.hdf5
Epoch 47/70
2793/2793 [==============================] - 29s 10ms/step - loss: 0.9091

Epoch 00047: loss improved from 1.01828 to 0.90908, saving model to weights-improvement-47-0.9091-bigger2.hdf5
Epoch 48/70
2793/2793 

In [31]:
from keras.models import load_model
model.save('modelo_idiota.h5')


In [61]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print( "Seed:")
print( "\"", ''.join([int_to_char[value] for value in pattern]), "\"")

Seed:
" nothing of tumbling down stairs! how brave they’ll all think me at
home! why, i wouldn’t say anythin "


In [84]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
palavra = 'rabb'
pattern = [char_to_int[value] for value in palavra]
print(pattern)
print(len(pattern))
print( "Seed:")
print( "\"", ''.join([int_to_char[value] for value in pattern]), "\"")

[34, 17, 18, 18]
4
Seed:
" rabb "


In [103]:
# generate characters
# Ler 5 anteriores para prever 1
# ir passo a passo vendo a melhoria gerada com o script do fil
for i in range(100):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np .argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print( "\nDone.")


ValueError: Error when checking input: expected lstm_5_input to have shape (100, 1) but got array with shape (4, 1)

In [100]:
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
  #  index = np .argmax(prediction)
  #result = int_to_char[index]


ValueError: Error when checking input: expected lstm_5_input to have shape (100, 1) but got array with shape (4, 1)